In [1]:
import tensorflow as tf
import tensorflow_hub as hub

import os


In [25]:
tf.__version__

'1.15.0'

In [22]:
hub.__version__

'0.7.0'

#### Defining cache directory for tf hub

In [3]:
os.environ["TFHUB_CACHE_DIR"] = '/space/engineering/tfhub_modules'

OUTPUT_PATH="/var/extra/users/jgeorge/tf2.0/USE/large"
use_output_path= os.path.join(OUTPUT_PATH,'v3')

In [4]:
embed_module = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
# module = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")


In [5]:
##restricting no of gpus
gpus = tf.config.experimental.list_physical_devices('GPU')
if len(gpus)>0:
    device_to_use = gpus[2]
    tf.config.experimental.set_memory_growth(device_to_use,True)
    tf.config.experimental.set_visible_devices(device_to_use, 'GPU')
    print(tf.config.experimental.get_visible_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]


In [6]:
sess = tf.Session()

In [7]:
text1 = tf.placeholder(tf.string,shape=(None),name="input_text1")
text2 = tf.placeholder(tf.string,shape=(None),name="input_text2")
embedding1 = embed_module(text1)
embedding2 = embed_module(text2)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [8]:
normalize_t1 = tf.nn.l2_normalize(embedding1,axis=-1) 
normalize_t2 = tf.nn.l2_normalize(embedding2,axis=-1) 

In [9]:
# https://stackoverflow.com/questions/43357732/how-to-calculate-the-cosine-similarity-between-two-tensors
# certain values where slightly above 1 (1+1e-5), clipping it a 1
similarity = tf.clip_by_value(
    tf.reduce_sum(tf.multiply(normalize_t1,normalize_t2),axis=-1),
    clip_value_min=0, clip_value_max=1,name='cosine_similarity')


In [10]:
init_op = tf.group([tf.global_variables_initializer(), 
tf.tables_initializer()])

In [11]:
sess.run(init_op)

### various functions this model would support

Running with equal no of inputs for both arguments

In [12]:
sess.run(similarity,feed_dict={text1:["talk to agent","talk to representative"],text2:["talk to agent","talk to rep"]})

array([1.       , 0.7762475], dtype=float32)

Running with single input as 1st argument & multiple inputs for 2nd argument

In [13]:
sess.run(similarity,feed_dict={text1:["talk to agent"],text2:["talk to agent","talk to rep"]})

array([1.       , 0.6674439], dtype=float32)

Fetching the embedding

In [14]:
sess.run(embedding1,feed_dict={text1:["talk to an agent"]})

array([[-3.80791053e-02,  2.50815903e-03, -9.76895727e-03,
        -6.32836437e-03, -6.88123703e-02,  5.86753264e-02,
        -5.80106676e-02,  1.19542992e-02, -4.40222919e-02,
        -6.84329262e-03,  2.51274072e-02, -2.81585604e-02,
         1.76772084e-02, -6.33776486e-02,  5.12930416e-02,
        -1.04572112e-02,  5.23131853e-03,  7.27704391e-02,
        -7.18747824e-02, -4.40347604e-02,  6.81014219e-03,
         2.03395505e-02,  1.81480851e-02,  4.99802548e-03,
         9.23127905e-02, -8.59296322e-03, -6.34736046e-02,
        -2.11174972e-02, -3.86463925e-02, -2.34054048e-02,
        -5.40573001e-02, -2.32361332e-02, -2.56250240e-02,
         5.02044484e-02, -2.73906421e-02, -5.36784902e-02,
         5.40216565e-02,  8.37131068e-02, -5.94174303e-02,
         3.37379649e-02, -3.53883021e-02, -4.74682339e-02,
        -2.07184926e-02, -4.76788469e-02,  6.27262741e-02,
         1.15935411e-02,  4.12412174e-02,  6.94981869e-03,
         4.64623347e-02,  6.58587366e-02,  6.01422563e-0

In [15]:
builder = tf.saved_model.builder.SavedModelBuilder(use_output_path)

### Signature defs
keeping 2 signatures-   
1) for returning just the embedding   
2) for cosine similarity

In [16]:

embedding_signature = tf.saved_model.predict_signature_def(inputs={'input_text':text1},
                                                           outputs={'embedding':embedding1})
similarity_signature = tf.saved_model.predict_signature_def(inputs={'input_text1':text1,'input_text2':text2},
                                                            outputs={'similarity':similarity})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [17]:
embedding_signature

inputs {
  key: "input_text"
  value {
    name: "input_text1:0"
    dtype: DT_STRING
    tensor_shape {
      unknown_rank: true
    }
  }
}
outputs {
  key: "embedding"
  value {
    name: "module_apply_default/Encoder_en/hidden_layers/l2_normalize:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 512
      }
    }
  }
}
method_name: "tensorflow/serving/predict"

In [18]:
similarity_signature

inputs {
  key: "input_text1"
  value {
    name: "input_text1:0"
    dtype: DT_STRING
    tensor_shape {
      unknown_rank: true
    }
  }
}
inputs {
  key: "input_text2"
  value {
    name: "input_text2:0"
    dtype: DT_STRING
    tensor_shape {
      unknown_rank: true
    }
  }
}
outputs {
  key: "similarity"
  value {
    name: "cosine_similarity:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
    }
  }
}
method_name: "tensorflow/serving/predict"

In [19]:
EMBEDDING_SIG_DEF_KEY='embedding'
SIMILARITY_SIG_DEF_KEY='similarity'

Keeping serving default as embedding signature itself

In [20]:
builder.add_meta_graph_and_variables(sess,tags=[tf.saved_model.tag_constants.SERVING],
                                     signature_def_map={
                                         tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY:embedding_signature,
                                                       EMBEDDING_SIG_DEF_KEY:embedding_signature,
                                                       SIMILARITY_SIG_DEF_KEY:similarity_signature},
                                     main_op=tf.group([tf.tables_initializer()]))
builder.save()

INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: /var/extra/users/jgeorge/tf2.0/USE/large/v3/saved_model.pb


INFO:tensorflow:SavedModel written to: /var/extra/users/jgeorge/tf2.0/USE/large/v3/saved_model.pb


b'/var/extra/users/jgeorge/tf2.0/USE/large/v3/saved_model.pb'

In [21]:
# instead of passing the variable to create the signature definition we can use this for passing tensorinfo, no longer required 
# tf.saved_model.build_tensor_info(embedding1)